In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import cv2

import tensorflow as tf
import numpy as np
from random import randint
import keras
from keras.datasets import mnist
import keras.backend as K
from keras.models import Model, Sequential
from keras.layers.merge import Concatenate
from keras.layers import Input, Reshape, Dense, Conv2D, Dropout, \
    MaxPooling2D, Flatten, UpSampling2D, Multiply, Activation, AveragePooling2D, \
    Add, Subtract, Lambda
from keras.callbacks import ModelCheckpoint, TensorBoard
from scipy.ndimage import gaussian_filter as blur
from keras.optimizers import Adam
from keras.utils import multi_gpu_model as mgpu
from keras.utils import np_utils
import os, sys

Using TensorFlow backend.


In [2]:
model = Sequential()
# model.add(Dense(512, input_shape=(28,28, 1), activation='relu'))
model.add(Conv2D(64, (3, 3), padding='same', activation='relu', 
                 input_shape=(28, 28, 1)))
model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D((2, 2), strides=(2, 2), padding='same')) # = 14

model.add(Conv2D(256, (3, 3), padding='same', activation='relu'))
model.add(Conv2D(256, (3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D((2, 2), strides=(2, 2), padding='same')) # =7

model.add(Conv2D(512, (3, 3), padding='same', activation='relu'))
model.add(Conv2D(512, (3, 3), padding='same', activation='relu'))

model.add(Reshape((7 * 7 * 512,)))
model.add(Dense(2048, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(2048, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(10))
model.add(Activation('softmax'))


In [3]:
nb_classes = 10
(X_train, y_train), (X_test, y_test) = mnist.load_data()
print("X_train original shape", X_train.shape)
print("y_train original shape", y_train.shape)
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)

# X_train = X_train.reshape(60000, 784)
# X_test = X_test.reshape(10000, 784)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255
print("Training matrix shape", X_train.shape)
print("Testing matrix shape", X_test.shape)

X_train original shape (60000, 28, 28)
y_train original shape (60000,)
Training matrix shape (60000, 28, 28)
Testing matrix shape (10000, 28, 28)


In [4]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 28, 28, 64)        640       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 28, 28, 128)       73856     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 128)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 14, 14, 256)       295168    
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 14, 14, 256)       590080    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 7, 7, 256)         0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 7, 7, 512)         1180160   
__________

In [5]:
Y_train[0]

array([0., 0., 0., 0., 0., 1., 0., 0., 0., 0.], dtype=float32)

In [13]:
import cv2
bsize = 64

def resize(img, to=64):
    scale = to / img.shape[0]
    return cv2.resize(img, (0,0), fx=scale, fy=scale)

def datagen():
    while True:
        for b0 in range(0, len(X_train), bsize):
            batch_imgs = X_train[b0:b0+bsize]
            batch_lbls = Y_train[b0:b0+bsize]
            if len(batch_lbls) < bsize: continue
    #         batch_imgs = [resize(img, 64) for img in batch_imgs]
            batch_imgs = np.array(batch_imgs).reshape((bsize, 28, 28, 1))
    #         assert batch_imgs[0].shape == (64, 64, 1)
            yield batch_imgs, batch_lbls
        
print(type(datagen()))

<class 'generator'>


In [14]:
hist = model.fit_generator(
    datagen(),
    epochs=4,
    verbose=1,
    steps_per_epoch=len(X_train)//bsize-1
#           validation_data=(X_test, Y_test)
)

Epoch 1/4
936/936 [==============================] - 43s 46ms/step - loss: 0.0363 - acc: 0.9896
Epoch 2/4
936/936 [==============================] - 43s 46ms/step - loss: 0.0314 - acc: 0.9913
Epoch 3/4
936/936 [==============================] - 43s 46ms/step - loss: 0.0279 - acc: 0.9920
Epoch 4/4
936/936 [==============================] - 43s 46ms/step - loss: 0.0251 - acc: 0.9933


In [ ]:
print(hist.history)

import matplotlib.pyplot as plt
plt.figure(figsize=(14, 5))
plt.plot(hist.history['loss'])
plt.show()
plt.close()